# Agent Component: Central Dispatcher

The `CentralDispatcher` is a high-level coordinating agent. It sits above local controllers and implements a system-wide strategy. It works by:

1. **Subscribing** to multiple state topics from across the system to build a global view of the situation.
2. **Evaluating** a set of rules or a strategy based on this global view.
3. **Publishing** high-level commands (like new setpoints) to local control agents to change their behavior.

This agent has a hybrid execution model:
- Its knowledge of the system state is updated in a purely **event-driven** way via message subscriptions.
- Its decision-making logic is executed in its `run()` method, which is called periodically by the simulation harness. This is a **time-stepped** model.

## Simulation Example

We will demonstrate the dispatcher's rule-based logic. We will configure it with a rule to issue a 'flood alert' command if a reservoir's water level goes above a certain threshold.

1. We will create a `CentralDispatcher` with this rule.
2. We will set up a listener to capture any commands it sends.
3. We will first run the agent with a normal water level.
4. Then, we will publish a high water level to the state topic and run the agent again. This should trigger the rule and cause a command to be published.

In [ ]:
import pandas as pd
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher
from swp.central_coordination.collaboration.message_bus import MessageBus

# 1. Setup components
message_bus = MessageBus()
STATE_TOPIC = "state.reservoir.level"
COMMAND_TOPIC = "command.gate1.setpoint"

# The rules define the dispatcher's behavior
rules = {
    'flood_threshold': 18.0,
    'normal_setpoint': 15.0,
    'flood_setpoint': 12.0
}

dispatcher = CentralDispatcher(
    agent_id="main_dispatcher",
    message_bus=message_bus,
    state_subscriptions={'reservoir_level': STATE_TOPIC},
    command_topics={'gate1_command': COMMAND_TOPIC},
    rules=rules
)

# 2. Create a listener for the dispatcher's commands
captured_commands = []
def command_listener(message):
    print(f"--> LISTENER CAPTURED COMMAND: {message}")
    captured_commands.append(message)

message_bus.subscribe(COMMAND_TOPIC, command_listener)

# 3. Simulation
print("--- Step 1: Initial Run ---")
dispatcher.run(current_time=0)
print("Dispatcher sends initial setpoint on first run.")

print("\n--- Step 2: Normal Conditions ---")
# Publish a normal state message
message_bus.publish(STATE_TOPIC, {'water_level': 16.0})
print("Published normal water level (16.0m). Running dispatcher...")
dispatcher.run(current_time=1)
print("No new command expected.")

print("\n--- Step 3: Flood Conditions ---")
# Publish a high water level message
message_bus.publish(STATE_TOPIC, {'water_level': 19.5})
print("Published high water level (19.5m). Running dispatcher...")
dispatcher.run(current_time=2)
print("Flood alert command expected.")

print("\n--- Simulation Finished ---")

## Results

Our listener captured two commands:
1. The initial 'normal' setpoint that the dispatcher sends on its first run.
2. The 'flood' setpoint, which was correctly triggered only after the water level was updated to be above the `flood_threshold`.

In [ ]:
print(f"Captured {len(captured_commands)} commands:")
for i, cmd in enumerate(captured_commands):
    print(f"  Command {i+1}: {cmd}")